In [1]:
library(text2vec)
library(glmnet)
library(slam)

Loading required package: Matrix

Loaded glmnet 4.1-4



### Split generation

In [1]:
data <- read.table("alldata.tsv", stringsAsFactors = FALSE,
                  header = TRUE)
testIDs <- read.csv("project3_splits.csv", header = TRUE)
for(j in 1:5){
  dir.create(paste("split_", j, sep=""))
  train <- data[-testIDs[,j], c("id", "sentiment", "review") ]
  test <- data[testIDs[,j], c("id", "review")]
  test.y <- data[testIDs[,j], c("id", "sentiment", "score")]
  
  tmp_file_name <- paste("split_", j, "/", "train.tsv", sep="")
  write.table(train, file=tmp_file_name, 
              quote=TRUE, 
              row.names = FALSE,
              sep='\t')
  tmp_file_name <- paste("split_", j, "/", "test.tsv", sep="")
  write.table(test, file=tmp_file_name, 
              quote=TRUE, 
              row.names = FALSE,
              sep='\t')
  tmp_file_name <- paste("split_", j, "/", "test_y.tsv", sep="")
  write.table(test.y, file=tmp_file_name, 
            quote=TRUE, 
            row.names = FALSE,
            sep='\t')
}

### Generate vocabulary

In [2]:
# load the training data and clean the html tags.
#j = 1
#setwd(paste("split_", j, sep=""))
#train = read.table("train.tsv",
train = read.table("alldata.tsv",
                   stringsAsFactors = FALSE,
                   header = TRUE)
train$review = gsub('<.*?>', ' ', train$review)

In [3]:
# construct DT (DocumentTerm) matrix (maximum 4-grams).
stop_words = c("i", "me", "my", "myself", 
               "we", "our", "ours", "ourselves", 
               "you", "your", "yours", 
               "their", "they", "his", "her", 
               "she", "he", "a", "an", "and",
               "is", "was", "are", "were", 
               "him", "himself", "has", "have", 
               "it", "its", "the", "us")
it_train = itoken(train$review,
                  preprocessor = tolower, 
                  tokenizer = word_tokenizer)
tmp.vocab = create_vocabulary(it_train, 
                              stopwords = stop_words, 
                              ngram = c(1L,4L))
tmp.vocab = prune_vocabulary(tmp.vocab, term_count_min = 10,
                             doc_proportion_max = 0.5,
                             doc_proportion_min = 0.001)
dtm_train  = create_dtm(it_train, vocab_vectorizer(tmp.vocab))

In [4]:
# use Lasso (with logistic regression) to trim the vocabulary size to 2K.
set.seed(7568)
tmpfit = glmnet(x = dtm_train, 
                y = train$sentiment, 
                alpha = 1,
                family='binomial')
tmpfit$df

[1]     0     1     2     3     4     4     6     7    11    15    18    22
 [13]    25    39    48    57    67    83    97   113   131   152   174   208
 [25]   238   272   303   337   388   437   489   558   638   740   857   976
 [37]  1125  1279  1455  1685  1934  2219  2529  2871  3232  3593  4011  4395
 [49]  4779  5167  5583  6022  6432  6820  7204  7548  7909  8241  8570  8867
 [61]  9117  9390  9651  9884 10112 10333 10543 10699 10847 10972 11105 11260
 [73] 11408 11510 11602 11708 11784 11876 11936 12004 12093 12149 12203 12266
 [85] 12307 12359 12406 12450 12476 12586 12647 12692 12734 12775 12811 12830
 [97] 12865 12896 12918 12941

In [5]:
# picke the largest df among the beta values thar are less than 2K 
#(which turns out to be the 44th column), and store the corresponding words in myvocab
myvocab = colnames(dtm_train)[which(tmpfit$beta[, 44] != 0)]

In [6]:
# use this customized vocabulary with ridge regression on the five splits
#train = read.table("train.tsv",
train = read.table("alldata.tsv",
                   stringsAsFactors = FALSE,
                   header = TRUE)
 train$review <- gsub('<.*?>', ' ', train$review)
 it_train = itoken(train$review,
                    preprocessor = tolower, 
                    tokenizer = word_tokenizer)
 vectorizer = vocab_vectorizer(create_vocabulary(myvocab, 
                                                  ngram = c(1L, 2L)))
 dtm_train = create_dtm(it_train, vectorizer)

In [7]:
# use Lasso (with logistic regression) to trim the vocabulary size to 2K.
v.size = dim(dtm_train)[2]
ytrain = train$sentiment

summ = matrix(0, nrow=v.size, ncol=4)
summ[,1] = colapply_simple_triplet_matrix(
  as.simple_triplet_matrix(dtm_train[ytrain==1, ]), mean)
summ[,2] = colapply_simple_triplet_matrix(
  as.simple_triplet_matrix(dtm_train[ytrain==1, ]), var)
summ[,3] = colapply_simple_triplet_matrix(
  as.simple_triplet_matrix(dtm_train[ytrain==0, ]), mean)
summ[,4] = colapply_simple_triplet_matrix(
  as.simple_triplet_matrix(dtm_train[ytrain==0, ]), var)

n1 = sum(ytrain); 
n = length(ytrain)
n0 = n - n1

myp = (summ[,1] - summ[,3])/
  sqrt(summ[,2]/n1 + summ[,4]/n0)

In [8]:
# order words by the magnitude of their t-statistics and
# pick the top 2000 words, which are then divided into two 
# lists: positive words and negative words.
words = colnames(dtm_train)
id = order(abs(myp), decreasing=TRUE)[1:2000]
pos.list = words[id[myp[id]>0]]
neg.list = words[id[myp[id]<0]]

In [22]:
pos.list[1:50]

[1] "great"              "excellent"          "wonderful"         
 [4] "best"               "love"               "perfect"           
 [7] "loved"              "amazing"            "beautiful"         
[10] "superb"             "well"               "favorite"          
[13] "brilliant"          "highly"             "life"              
[16] "must_see"           "also"               "fantastic"         
[19] "very"               "beautifully"        "one_of"            
[22] "both"               "today"              "always"            
[25] "very_well"          "performance"        "enjoyed"           
[28] "performances"       "well_worth"         "highly_recommend"  
[31] "years"              "8_10"               "touching"          
[34] "10_10"              "wonderfully"        "outstanding"       
[37] "world"              "perfectly"          "strong"            
[40] "highly_recommended" "7_10"               "powerful"          
[43] "8"                  "definitely"         "on_dvd"            
[46] "terrific"           "classic"            "unique"            
[49] "especially"         "love_this"

In [21]:
neg.list[1:50]

[1] "bad"         "worst"       "waste"       "awful"       "terrible"   
 [6] "worse"       "boring"      "no"          "stupid"      "nothing"    
[11] "waste_of"    "poor"        "horrible"    "minutes"     "even"       
[16] "so_bad"      "just"        "crap"        "supposed"    "acting"     
[21] "poorly"      "at_all"      "why"         "plot"        "script"     
[26] "ridiculous"  "lame"        "worst_movie" "avoid"       "not_even"   
[31] "wasted"      "annoying"    "don't"       "oh"          "dull"       
[36] "mess"        "pointless"   "money"       "laughable"   "any"        
[41] "cheap"       "pathetic"    "least"       "thing"       "badly"      
[46] "unless"      "instead"     "bad_acting"  "could"       "1"

In [15]:
# check which words never appear in the positive reviews 
# and which never appear in the negative reviews.
id1 = which(summ[, 2] == 0) # same as: which(summ[id0, 1] != 0)
id0 = which(summ[, 4] == 0) #same as: which(summ[id1, 3] != 0)
words[id1]
words[id0]

[1] "10_out_of"                    "10_out_of_10"                
  [3] "1_2_from"                     "1_out_of"                    
  [5] "2_out_of_10"                  "3_out_of_10"                 
  [7] "4_out_of_10"                  "7_out_of"                    
  [9] "7_out_of_10"                  "8_out_of"                    
 [11] "9_out_of_10"                  "able_to_see"                 
 [13] "about_woman"                  "any_of_characters"           
 [15] "anyone_interested_in"         "as_good_as"                  
 [17] "as_know"                      "as_much_as"                  
 [19] "as_should_be"                 "at_height"                   
 [21] "at_own"                       "at_times_but"                
 [23] "bad_as_this"                  "bad_at_all"                  
 [25] "based_on_real"                "based_on_true_story"         
 [27] "be_better_off"                "be_bit"                      
 [29] "be_forced_to"                 "be_seen_by"                  
 [31] "been_fan_of"                  "best_can"                    
 [33] "best_movie_ever"              "best_of_all"                 
 [35] "best_thing_about_this"        "both_of_which"               
 [37] "bottom_of_barrel"             "but_do_not"                  
 [39] "but_in_this"                  "but_no_one"                  
 [41] "but_not_much"                 "but_script"                  
 [43] "but_there_no"                 "but_to_say"                  
 [45] "by_any_means"                 "case_in_point"               
 [47] "chance_to_see"                "come_close_to"               
 [49] "come_to_think"                "comes_off_as"                
 [51] "could_been"                   "could_been_good"             
 [53] "day_to_day"                   "didn't_know_what"            
 [55] "do_not_recommend"             "do_not_watch_this"           
 [57] "do_yourself_favor"            "does_not_make"               
 [59] "don't_care_about"             "don't_know_why"              
 [61] "don't_think_so"               "don't_watch_this"            
 [63] "even_close_to"                "every_time_watch"            
 [65] "example_of_how"               "far_too_much"                
 [67] "few_years_later"              "film_did_not"                
 [69] "films_of_this"                "first_saw_this"              
 [71] "for_all_ages"                 "for_anyone_who"              
 [73] "for_first_time"               "forced_to_watch"             
 [75] "forward_to_this"              "from_outer_space"            
 [77] "from_start_to_finish"         "get_copy"                    
 [79] "give_1"                       "give_3"                      
 [81] "give_4"                       "give_break"                  
 [83] "give_this_movie"              "glad_did"                    
 [85] "go_see_this"                  "going_to_see"                
 [87] "had_no_idea"                  "happened_to_be"              
 [89] "heart_of_gold"                "highly_recommend_this"       
 [91] "hope_to_see"                  "hours_of_life"               
 [93] "how_not_to"                   "i'm_big"                     
 [95] "if_aren't"                    "if_can"                      
 [97] "if_did"                       "if_do_not"                   
 [99] "if_don't_like"                "if_had"                      
[101] "if_haven't"                   "if_know"                     
[103] "if_must"                      "if_really"                   
[105] "if_want_good"                 "if_want_to"                  
[107] "if_want_to_see"               "if_watching"                 
[109] "if_you've_ever"               "in_entire_life"              
[111] "in_mouth"                     "in_of_itself"                
[113] "in_right_place"               "in_some_of"                  
[115] "in_world_war"                 "interest_in_this"            
[117] "it's_movie"                   "joy_to_watch"       

[1] "10_out_of"                    "10_out_of_10"                
  [3] "1_2_from"                     "1_out_of"                    
  [5] "2_out_of_10"                  "3_out_of_10"                 
  [7] "4_out_of_10"                  "7_out_of"                    
  [9] "7_out_of_10"                  "8_out_of"                    
 [11] "9_out_of_10"                  "able_to_see"                 
 [13] "again_again"                  "any_of_characters"           
 [15] "anyone_interested_in"         "as_good_as"                  
 [17] "as_know"                      "as_much_as"                  
 [19] "as_should_be"                 "at_height"                   
 [21] "at_right"                     "at_same"                     
 [23] "at_times_but"                 "back_enjoy"                  
 [25] "bad_as_this"                  "bad_at_all"                  
 [27] "based_on_real"                "based_on_true_story"         
 [29] "be_better_off"                "be_bit"                      
 [31] "be_forced_to"                 "be_seen_by"                  
 [33] "been_fan_of"                  "best_movie_ever"             
 [35] "best_of_all"                  "best_thing_about_this"       
 [37] "both_of_which"                "bottom_of_barrel"            
 [39] "but_do_not"                   "but_in_this"                 
 [41] "but_liked"                    "but_no_one"                  
 [43] "but_not_much"                 "but_there_no"                
 [45] "but_to_say"                   "by_any_means"                
 [47] "case_in_point"                "cast_excellent"              
 [49] "chance_to_see"                "come_close_to"               
 [51] "come_to_think"                "comes_off_as"                
 [53] "could_been_good"              "day_to_day"                  
 [55] "didn't_know_what"             "do_not_recommend"            
 [57] "do_not_watch_this"            "do_yourself_favor"           
 [59] "does_not_make"                "don't_care_about"            
 [61] "don't_know_why"               "don't_think_so"              
 [63] "don't_watch_this"             "even_close_to"               
 [65] "every_time_watch"             "example_of_how"              
 [67] "far_too_much"                 "few_years_later"             
 [69] "film_did_not"                 "films_of_this"               
 [71] "first_saw_this"               "for_all_ages"                
 [73] "for_anyone_who"               "for_first_time"              
 [75] "forced_to_watch"              "forward_to_this"             
 [77] "from_outer_space"             "from_start_to_finish"        
 [79] "get_copy"                     "give_7"                      
 [81] "give_break"                   "give_this_movie"             
 [83] "glad_did"                     "go_see_this"                 
 [85] "going_to_see"                 "had_no_idea"                 
 [87] "happened_to_be"               "heart_of_gold"               
 [89] "highly_recommend_this"        "hope_to_see"                 
 [91] "hours_of_life"                "how_not_to"                  
 [93] "if_aren't"                    "if_do_not"                   
 [95] "if_don't"                     "if_don't_like"               
 [97] "if_haven't"                   "if_really"                   
 [99] "if_want_good"                 "if_want_to"                  
[101] "if_want_to_see"               "if_you've_ever"              
[103] "in_entire_life"               "in_of_itself"                
[105] "in_right_place"               "in_some_of"                  
[107] "in_streets"                   "in_world_war"                
[109] "interest_in_this"             "joy_to_watch"                
[111] "just_as_good"                 "just_seems_to"               
[113] "know_what_to_expect"          "last_20_minutes"             
[115] "laugh_out_loud"               "let_know"                    
[117] "like_no_other"                "little_to_do"       

In [16]:
words[id0[! (id0 %in% id)]]
words[id1[! (id1 %in% id)]]

[1] "10_out_of"                    "10_out_of_10"                
  [3] "1_2_from"                     "1_out_of"                    
  [5] "2_out_of_10"                  "3_out_of_10"                 
  [7] "4_out_of_10"                  "7_out_of"                    
  [9] "7_out_of_10"                  "8_out_of"                    
 [11] "9_out_of_10"                  "able_to_see"                 
 [13] "again_again"                  "any_of_characters"           
 [15] "anyone_interested_in"         "as_good_as"                  
 [17] "as_know"                      "as_much_as"                  
 [19] "as_should_be"                 "at_height"                   
 [21] "at_right"                     "at_same"                     
 [23] "at_times_but"                 "back_enjoy"                  
 [25] "bad_as_this"                  "bad_at_all"                  
 [27] "based_on_real"                "based_on_true_story"         
 [29] "be_better_off"                "be_bit"                      
 [31] "be_forced_to"                 "be_seen_by"                  
 [33] "been_fan_of"                  "best_movie_ever"             
 [35] "best_of_all"                  "best_thing_about_this"       
 [37] "both_of_which"                "bottom_of_barrel"            
 [39] "but_do_not"                   "but_in_this"                 
 [41] "but_liked"                    "but_no_one"                  
 [43] "but_not_much"                 "but_there_no"                
 [45] "but_to_say"                   "by_any_means"                
 [47] "case_in_point"                "cast_excellent"              
 [49] "chance_to_see"                "come_close_to"               
 [51] "come_to_think"                "comes_off_as"                
 [53] "could_been_good"              "day_to_day"                  
 [55] "didn't_know_what"             "do_not_recommend"            
 [57] "do_not_watch_this"            "do_yourself_favor"           
 [59] "does_not_make"                "don't_care_about"            
 [61] "don't_know_why"               "don't_think_so"              
 [63] "don't_watch_this"             "even_close_to"               
 [65] "every_time_watch"             "example_of_how"              
 [67] "far_too_much"                 "few_years_later"             
 [69] "film_did_not"                 "films_of_this"               
 [71] "first_saw_this"               "for_all_ages"                
 [73] "for_anyone_who"               "for_first_time"              
 [75] "forced_to_watch"              "forward_to_this"             
 [77] "from_outer_space"             "from_start_to_finish"        
 [79] "get_copy"                     "give_7"                      
 [81] "give_break"                   "give_this_movie"             
 [83] "glad_did"                     "go_see_this"                 
 [85] "going_to_see"                 "had_no_idea"                 
 [87] "happened_to_be"               "heart_of_gold"               
 [89] "highly_recommend_this"        "hope_to_see"                 
 [91] "hours_of_life"                "how_not_to"                  
 [93] "if_aren't"                    "if_do_not"                   
 [95] "if_don't"                     "if_don't_like"               
 [97] "if_haven't"                   "if_really"                   
 [99] "if_want_good"                 "if_want_to"                  
[101] "if_want_to_see"               "if_you've_ever"              
[103] "in_entire_life"               "in_of_itself"                
[105] "in_right_place"               "in_some_of"                  
[107] "in_streets"                   "in_world_war"                
[109] "interest_in_this"             "joy_to_watch"                
[111] "just_as_good"                 "just_seems_to"               
[113] "know_what_to_expect"          "last_20_minutes"             
[115] "laugh_out_loud"               "let_know"                    
[117] "like_no_other"                "little_to_do"       

[1] "10_out_of"                    "10_out_of_10"                
  [3] "1_2_from"                     "1_out_of"                    
  [5] "2_out_of_10"                  "3_out_of_10"                 
  [7] "4_out_of_10"                  "7_out_of"                    
  [9] "7_out_of_10"                  "8_out_of"                    
 [11] "9_out_of_10"                  "able_to_see"                 
 [13] "about_woman"                  "any_of_characters"           
 [15] "anyone_interested_in"         "as_good_as"                  
 [17] "as_know"                      "as_much_as"                  
 [19] "as_should_be"                 "at_height"                   
 [21] "at_own"                       "at_times_but"                
 [23] "bad_as_this"                  "bad_at_all"                  
 [25] "based_on_real"                "based_on_true_story"         
 [27] "be_better_off"                "be_bit"                      
 [29] "be_forced_to"                 "be_seen_by"                  
 [31] "been_fan_of"                  "best_can"                    
 [33] "best_movie_ever"              "best_of_all"                 
 [35] "best_thing_about_this"        "both_of_which"               
 [37] "bottom_of_barrel"             "but_do_not"                  
 [39] "but_in_this"                  "but_no_one"                  
 [41] "but_not_much"                 "but_script"                  
 [43] "but_there_no"                 "but_to_say"                  
 [45] "by_any_means"                 "case_in_point"               
 [47] "chance_to_see"                "come_close_to"               
 [49] "come_to_think"                "comes_off_as"                
 [51] "could_been"                   "could_been_good"             
 [53] "day_to_day"                   "didn't_know_what"            
 [55] "do_not_recommend"             "do_not_watch_this"           
 [57] "do_yourself_favor"            "does_not_make"               
 [59] "don't_care_about"             "don't_know_why"              
 [61] "don't_think_so"               "don't_watch_this"            
 [63] "even_close_to"                "every_time_watch"            
 [65] "example_of_how"               "far_too_much"                
 [67] "few_years_later"              "film_did_not"                
 [69] "films_of_this"                "first_saw_this"              
 [71] "for_all_ages"                 "for_anyone_who"              
 [73] "for_first_time"               "forced_to_watch"             
 [75] "forward_to_this"              "from_outer_space"            
 [77] "from_start_to_finish"         "get_copy"                    
 [79] "give_1"                       "give_3"                      
 [81] "give_4"                       "give_break"                  
 [83] "give_this_movie"              "glad_did"                    
 [85] "go_see_this"                  "going_to_see"                
 [87] "had_no_idea"                  "happened_to_be"              
 [89] "heart_of_gold"                "highly_recommend_this"       
 [91] "hope_to_see"                  "hours_of_life"               
 [93] "how_not_to"                   "i'm_big"                     
 [95] "if_aren't"                    "if_can"                      
 [97] "if_did"                       "if_do_not"                   
 [99] "if_don't_like"                "if_had"                      
[101] "if_haven't"                   "if_know"                     
[103] "if_must"                      "if_really"                   
[105] "if_want_good"                 "if_want_to"                  
[107] "if_want_to_see"               "if_you've_ever"              
[109] "in_entire_life"               "in_mouth"                    
[111] "in_of_itself"                 "in_right_place"              
[113] "in_some_of"                   "in_world_war"                
[115] "interest_in_this"             "it's_movie"                  
[117] "joy_to_watch"                 "just_as_good"       

In [30]:
# Write vocab to text file
some.strs <- c(words)
write.table(some.strs, file = "myvocab.txt",
            quote = FALSE,
            row.names = FALSE,
            col.names = FALSE,
            sep = "\n")

### Submitted code

In [69]:
#myvocab <- scan(file = "myvocab.txt", what = character())
#j=1
#setwd(paste("split_", j, sep=""))
train = read.table("train.tsv",
                   stringsAsFactors = FALSE,
                   header = TRUE)
 train$review <- gsub('<.*?>', ' ', train$review)

# Create matrix corresponding to vocab
 it_train = itoken(train$review,
                    preprocessor = tolower, 
                    tokenizer = word_tokenizer)
 vectorizer = vocab_vectorizer(create_vocabulary(myvocab, 
                                                  ngram = c(1L, 2L)))
 dtm_train = create_dtm(it_train, vectorizer)

In [70]:
set.seed(7568)
tmpfit = glmnet(x = dtm_train, 
                y = train$sentiment, 
                alpha = 0.01,
                family='binomial')
tmpfit$df

[1]    0    1    2    4    5    5    9   16   21   34   50   65   85  113  152
 [16]  191  238  281  335  399  471  531  597  661  745  839  930 1028 1122 1230
 [31] 1340 1436 1526 1628 1727 1831 1904 1978 2033 2096 2147 2193 2237 2278 2307
 [46] 2342 2362 2383 2403 2416 2433 2446 2459 2466 2478 2483 2487 2497 2502 2505
 [61] 2508 2509 2511 2518 2520 2524 2522 2525 2523 2527 2527 2532 2530 2528 2528
 [76] 2527 2529 2524 2527 2533 2540 2545 2546 2547 2552 2557 2559 2555 2557 2553
 [91] 2554 2555 2556 2562 2567 2570 2569 2571 2572 2571

In [71]:
#####################################
# Load test data, and 
# Compute prediction
#####################################
test <- read.table("test.tsv", stringsAsFactors = FALSE,
                    header = TRUE)

 test$review <- gsub('<.*?>', ' ', test$review)

# Create matrix corresponding to vocab
 it_test = itoken(test$review,
                    preprocessor = tolower, 
                    tokenizer = word_tokenizer)
 vectorizer = vocab_vectorizer(create_vocabulary(myvocab, 
                                                  ngram = c(1L, 2L)))
 dtm_test = create_dtm(it_test, vectorizer)

In [72]:
predict(tmpfit, newx=dtm_test, s=0.01)

,s1
1,1.5828108
2,-3.6165582
3,0.1407497
4,3.5482136
5,-11.3678062
6,-6.1666739
7,2.7572158
8,-1.4156575
9,4.2640076
10,-16.9599254
